In [71]:
import gurobipy as gp
from itertools import product
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
def division(a,b):
    if a == 0:
        return 0
    if b == 0:
        return 0
    return a/b

In [2]:
# Create a new model
m = gp.Model()

# Create variables
x = m.addVar(vtype='B', name="x")
y = m.addVar(vtype='B', name="y")
z = m.addVar(vtype='B', name="z")

# Set objective function
m.setObjective(x + y + 2 * z, gp.GRB.MAXIMIZE)

# Add constraints
m.addConstr(x + 2 * y + 3 * z <= 4)
m.addConstr(x + y >= 1)

# Solve it!
m.optimize()

print(f"Optimal objective value: {m.objVal}")
print(f"Solution values: x={x.X}, y={y.X}, z={z.X}")

Set parameter TokenServer to value "dev.cma.mines-paristech.fr"
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x98886187
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
Optimal objective value: 3.0
Solution values: x=1.0, y=0.0, z=1.0


## 1 Modèle de base des centrales thermiques

### 1.1 Planification journalière du parc thermique

Variables de décision :
Pour une type de centrale $X \in [A,B,C]$, et une heure de la journée $t\in {1,\dots,24}$, on définit :
$$
N_t^{(X)} = \text{Nombre de centrales } X \text{allumées à } t\text{ h, (ENTIER)}
$$
$$
P_t^{(X)} = \text{Puissance totale produite par les centrales } X \text{ à } t\text{ h, (CONTINUE)}
$$
Contraintes :
$$
N_t^{(X)} P_{min}^{(X)} \leq P_t^{(X)} \leq N_t^{(X)} P_{max}^{(X)} \text{ ,  Contraintes sur la puissance totale de chaque centrale}
$$
$$
0 \leq N_t^{(X)} \leq N^{(X)} \text{   ,   Contraintes sur le nombre de centrales allumées possible}
$$
$$
\forall t, \sum_X P_t^{(X)} = D_t \text{  ,  Contrainte équilibre offre-demande}
$$
Objectif :
$$
\text{Minimiser} \sum_X \sum_T P_t^{(X)} C_{MWh}^{(X)}
$$

In [3]:
class Centrale:
    def __init__(self, name, N, Pmin, Pmax, Cmwh):
        self.name = name
        self.N = N
        self.Pmin = Pmin
        self.Pmax = Pmax
        self.Cmwh = Cmwh

In [68]:
for X in dict_11:
    print(dict_11[X].Pmin*dict_11[X].Cmwh)

1275.0
1724.9999999999998
4125.0


In [4]:
dict_11 = {}
dict_11["A"] = Centrale(
    name="A",
    N=12,
    Pmax=2000,
    Pmin=850,
    Cmwh=1.5
)
dict_11["B"] = Centrale(
    name="B",
    N=10,
    Pmax=1750,
    Pmin=1250,
    Cmwh=1.38
)
dict_11["C"] = Centrale(
    name="C",
    N=5,
    Pmax=4000,
    Pmin=1500,
    Cmwh=2.75
)
consommation = np.array([15,15,15,15,15,15,30,30,30,25,25,25,25,25,25,40,40,40,27,27,27,27,27,27])*1000

In [20]:
model = gp.Model(name="1.1")

In [21]:
dict_N = {}
dict_P = {}

for t in range(24):
    for X in dict_11:
        dict_N[X,t] = model.addVar(lb=0,ub=dict_11[X].N,vtype=gp.GRB.INTEGER,name=f"Nombre de centrale {X} allumées à {t}h")
    for X in dict_11:
        dict_P[X,t] = model.addVar(name=f"Puissance totale {X} à {t}h")


In [22]:

model.setObjective(gp.quicksum([dict_P[element[0],element[1]] * dict_11[element[0]].Cmwh for element in product(dict_11,range(24))]), gp.GRB.MINIMIZE)

In [23]:
for t in range(24):
    for X in dict_11:
        model.addConstr(dict_P[X,t]<=dict_N[X,t] * dict_11[X].Pmax, name=f"borne sup puissance, {X} à {t}h")
        model.addConstr(dict_P[X,t]>=dict_N[X,t] * dict_11[X].Pmin, name=f"borne inf puissance, {X} à {t}h")
    model.addConstr(gp.quicksum([dict_P[X,t] for X in dict_11])==consommation[t])

In [24]:
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 168 rows, 144 columns and 360 nonzeros
Model fingerprint: 0xe0a5527a
Variable types: 72 continuous, 72 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 3e+00]
  Bounds range     [5e+00, 1e+01]
  RHS range        [2e+04, 4e+04]
Found heuristic solution: objective 1235375.0000
Presolve removed 162 rows and 139 columns
Presolve time: 0.01s
Presolved: 6 rows, 5 columns, 14 nonzeros
Found heuristic solution: objective 881275.00000
Variable types: 2 continuous, 3 integer (0 binary)

Root relaxation: objective 8.694000e+05, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 869400.000    0    1 881275.000 869400.000  1.35%     -    

In [25]:
print(f"Coût : {model.ObjVal}")

Coût : 869400.0


In [73]:
def df_results(dict_N,dict_P,dict_XX):
    df = pd.DataFrame()
    df["h"] = range(24) 
    df["Consommation (MW)"] = consommation
    df["Production total"] = 0
    df["Coût total"] = 0
    for X in dict_XX:
        df[f"Nb centrale {X}"] = [int(dict_N[X,t].X) for t in range(24)]
        df[f"Puissance tot {X}"] = [dict_P[X,t].X for t in range(24)]
        df[f"Coût {X}"] = [dict_P[X,t].X * dict_XX[X].Cmwh for t in range(24)]
        df[f"Facteur de charge{X}"] = [int(division(dict_P[X,t].X ,dict_N[X,t].X * dict_11[X].Pmax)*100) for t in range(24)]
        df["Production total"] += df[f"Puissance tot {X}"]
        df["Coût total"] += df[f"Coût {X}"]
    df["Coût MWh"] = df["Coût total"]/df["Production total"]
    return df
df = df_results(dict_N,dict_P,dict_11)
df

,h,Consommation (MW),Production total,Coût total,Nb centrale A,Puissance tot A,Coût A,Facteur de chargeA,Nb centrale B,Puissance tot B,Coût B,Facteur de chargeB,Nb centrale C,Puissance tot C,Coût C,Facteur de chargeC,Coût MWh
0,0,15000,15000.0,21660.0,4,8000.0,12000.0,100,4,7000.0,9660.0,100,0,0.0,0.0,0,1.444000
1,1,15000,15000.0,21660.0,4,8000.0,12000.0,100,4,7000.0,9660.0,100,0,0.0,0.0,0,1.444000
2,2,15000,15000.0,21660.0,4,8000.0,12000.0,100,4,7000.0,9660.0,100,0,0.0,0.0,0,1.444000
3,3,15000,15000.0,21660.0,4,8000.0,12000.0,100,4,7000.0,9660.0,100,0,0.0,0.0,0,1.444000
4,4,15000,15000.0,21660.0,4,8000.0,12000.0,100,4,7000.0,9660.0,100,0,0.0,0.0,0,1.444000
5,5,15000,15000.0,21660.0,4,8000.0,12000.0,100,4,7000.0,9660.0,100,0,0.0,0.0,0,1.444000
6,6,30000,30000.0,43320.0,8,16000.0,24000.0,100,8,14000.0,19320.0,100,0,0.0,0.0,0,1.444000
7,7,30000,30000.0,43320.0,8,16000.0,24000.0,100,8,14000.0,19320.0,100,0,0.0,0.0,0,1.444000
8,8,30000,30000.0,43320.0,8,16000.0,24000.0,100,8,14000.0,19320.0,100,0,0.0,0.0,0,1.444000
9,9,25000,25000.0,35400.0,4,7500.0,11250.0,93,10,17500.0,24150.0,100,0,0.0,0.0,0,1.416000


In [47]:
from turtle import color
import plotly.graph_objects as go

fig = go.Figure()

Y = df["Consommation (MW)"].copy()*0
for X in dict_11:
    Y += df[f"Puissance tot {X}"]
    fig.add_trace(
        go.Scatter(
        x=df["h"], 
        y=Y, 
        fill='tonexty',
        name = f"Production {X}"
        )
    )

# fig.add_trace(
#     go.Scatter(
#         x = df["h"],
#         y = df["Consommation (MW)"],
#         name = "Consommation"
#     )
# )
fig.add_trace(
    go.Scatter(
        x = df["h"],
        y = df["Production total"],
        name="Production total",
        line = dict(dash='dash',color="red")
    )
)
fig.add_trace(
    go.Scatter(
        x=df["h"],
        y=df["Coût MWh"],
        yaxis="y2",
        name="Coût MWh",
        line_color="black"
    )
)

fig.update_layout(
    hovermode='x',
    yaxis=dict(title="MW" ,range=[0,df["Consommation (MW)"].max()*1.1]),
    yaxis2=dict(title="€/MWh",
    range=[df["Coût MWh"].min()*0.95,df["Coût MWh"].max()*1.1],
    anchor="free",
    overlaying="y",
    side="right",
    position=1
    ),
    title = "Répartition de la production électrique dans la journée"
)
fig
fig.show()

In [29]:
model_relax = model.relax()
model_relax.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 168 rows, 144 columns and 360 nonzeros
Model fingerprint: 0x48de0531
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 3e+00]
  Bounds range     [5e+00, 1e+01]
  RHS range        [2e+04, 4e+04]
Presolve removed 153 rows and 99 columns
Presolve time: 0.01s
Presolved: 15 rows, 45 columns, 45 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.5266000e+05   1.743750e+04   0.000000e+00      0s
      15    8.6940000e+05   0.000000e+00   0.000000e+00      0s

Solved in 15 iterations and 0.01 seconds (0.00 work units)
Optimal objective  8.694000000e+05


In [30]:
print(f"Coût : {model_relax.ObjVal}")

Coût : 869400.0


## 2 Coût d'opération

### 2.1 Coût de fonctionnement

Pour ce problème on modifie la fonction objectif.<br>
Objectif :
$$
\text{Minimiser} \sum_X \sum_T (P_t^{(X)} - P_{min}^{(X)}N_t^{(X)}) C_{MWh}^{(X)} + N_t^{(X)}C_{base}^{(X)}
$$

In [76]:
class Centrale2:
    def __init__(self, name, N, Pmin, Pmax, Cstart, Cbase, Cmwh):
        self.name = name
        self.N = N
        self.Pmin = Pmin
        self.Pmax = Pmax
        self.Cstart = Cstart
        self.Cbase = Cbase
        self.Cmwh = Cmwh

In [77]:
dict_2 = {}
dict_2["A"] = Centrale2(
    name="A",
    N=12,
    Pmax=2000,
    Pmin=850,
    Cmwh=2,
    Cstart=2000,
    Cbase=1000
)
dict_2["B"] = Centrale2(
    name="B",
    N=10,
    Pmax=1750,
    Pmin=1250,
    Cmwh=1.3,
    Cstart=1000,
    Cbase=2600
)
dict_2["C"] = Centrale2(
    name="C",
    N=5,
    Pmax=4000,
    Pmin=1500,
    Cmwh=3,
    Cstart=500,
    Cbase=3000
)

In [78]:
model = gp.Model(name="2.2")

dict_N = {}
dict_P = {}

for t in range(24):
    for X in dict_2:
        dict_N[X,t] = model.addVar(lb=0,ub=dict_2[X].N,vtype=gp.GRB.INTEGER,name=f"Nombre de centrale {X} allumées à {t}h")
    for X in dict_2:
        dict_P[X,t] = model.addVar(name=f"Puissance supplémentaire {X} à {t}h",vtype=gp.GRB.CONTINUOUS)

model.setObjective(gp.quicksum([dict_N[element[0],element[1]] * (dict_2[element[0]].Cbase - dict_2[element[0]].Pmin) + dict_P[element[0],element[1]] * dict_2[element[0]].Cmwh for element in product(dict_2,range(24))]), gp.GRB.MINIMIZE)

for t in range(24):
    for X in dict_11:
        model.addConstr(dict_P[X,t]<=dict_N[X,t] * dict_2[X].Pmax, name=f"borne sup puissance, {X} à {t}h")
        model.addConstr(dict_P[X,t]>=dict_N[X,t] * dict_2[X].Pmin, name=f"borne inf puissance, {X} à {t}h")
    model.addConstr(gp.quicksum([dict_P[X,t] for X in dict_2])==consommation[t])
    # model.addConstr(gp.quicksum([dict_P[X,t] for X in dict_11])==consommation[t])
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 168 rows, 144 columns and 360 nonzeros
Model fingerprint: 0x05c3801a
Variable types: 72 continuous, 72 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 2e+03]
  Bounds range     [5e+00, 1e+01]
  RHS range        [2e+04, 4e+04]
Found heuristic solution: objective 1626950.0000
Presolve removed 162 rows and 139 columns
Presolve time: 0.03s
Presolved: 6 rows, 5 columns, 14 nonzeros
Found heuristic solution: objective 1282700.0000
Variable types: 2 continuous, 3 integer (0 binary)

Root relaxation: objective 1.269413e+06, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1269412.50    0    1 1282700.00 1269412.50  1.04%     -    

In [59]:
print(f"Coût : {model.ObjVal}")

Coût : 1269450.0


In [53]:
def df_results2(dict_N,dict_P,dict_XX):
    df = pd.DataFrame()
    df["h"] = range(24) 
    df["Consommation (MW)"] = consommation
    df["Production total"] = 0
    df["Coût total"] = 0
    for X in dict_XX:
        df[f"Nb centrale {X}"] = [int(dict_N[X,t].X) for t in range(24)]
        df[f"Puissance tot {X}"] = [dict_P[X,t].X for t in range(24)]
        df[f"Coût {X}"] = [dict_N[X,t].X * dict_XX[X].Cbase + (dict_P[X,t].X - dict_N[X,t].X * dict_XX[X].Pmin) * dict_XX[X].Cmwh for t in range(24)]
        df["Production total"] += df[f"Puissance tot {X}"]
        df["Coût total"] += df[f"Coût {X}"]
    df["Coût MWh"] = df["Coût total"]/df["Production total"]
    return df
df_results2(dict_N=dict_N,dict_P=dict_P,dict_XX=dict_2)

,h,Consommation (MW),Production total,Coût total,Nb centrale A,Puissance tot A,Coût A,Nb centrale B,Puissance tot B,Coût B,Nb centrale C,Puissance tot C,Coût C,Coût MWh
0,0,15000,15000.0,26200.0,4,8000.0,13200.0,4,7000.0,13000.0,0,0.0,0.0,1.746667
1,1,15000,15000.0,26200.0,4,8000.0,13200.0,4,7000.0,13000.0,0,0.0,0.0,1.746667
2,2,15000,15000.0,26200.0,4,8000.0,13200.0,4,7000.0,13000.0,0,0.0,0.0,1.746667
3,3,15000,15000.0,26200.0,4,8000.0,13200.0,4,7000.0,13000.0,0,0.0,0.0,1.746667
4,4,15000,15000.0,26200.0,4,8000.0,13200.0,4,7000.0,13000.0,0,0.0,0.0,1.746667
5,5,15000,15000.0,26200.0,4,8000.0,13200.0,4,7000.0,13000.0,0,0.0,0.0,1.746667
6,6,30000,30000.0,52400.0,8,16000.0,26400.0,8,14000.0,26000.0,0,0.0,0.0,1.746667
7,7,30000,30000.0,52400.0,8,16000.0,26400.0,8,14000.0,26000.0,0,0.0,0.0,1.746667
8,8,30000,30000.0,52400.0,8,16000.0,26400.0,8,14000.0,26000.0,0,0.0,0.0,1.746667
9,9,25000,25000.0,44700.0,4,7500.0,12200.0,10,17500.0,32500.0,0,0.0,0.0,1.788000


### 2.2 Coût du démarrage

Pour ce problème on modifie les variables de décisions et on y adapte les contraintes.

Variables de décision :
Pour une type de centrale $X \in [A,B,C]$, et une heure de la journée $t\in {1,\dots,24}$, on définit :
$$
N_t^{(X)} = \text{Nombre de centrales } X \text{allumées à } t\text{ h, (ENTIER)}
$$
$$
N_{start,t}^{(X)} = \text{Nombre de centrales } X \text{démarrées à } t\text{ h, (ENTIER)}
$$
$$
P_t^{(X)} = \text{Puissance totale produite par les centrales } X \text{ à } t\text{ h, (CONTINUE)}
$$
Contraintes (par convention $N_{-1}^{(X)}=0$):
$$
N_t^{(X)} P_{min}^{(X)} \leq P_t^{(X)} \leq N_t^{(X)} P_{max}^{(X)} \text{ ,  Contraintes sur la puissance totale de chaque centrale}
$$
$$
N_{start,t}^{(X)} \leq N_t^{(X)} \leq N_{start,t}^{(X)}+N_{t-1}^{(X)} \text{   ,   Contraintes sur le nombre de centrales allumées possible}
$$
$$
0 \leq N_{start,t}^{(X)} \leq N^{(X)}-N_{t-1}^{(X)} \text{   ,   Contraintes sur le nombre de centrales adémarrées possible}
$$
$$
\sum_X P_t^{(X)} = D_t \text{  ,  Contrainte équilibre offre-demande}
$$
Objectif :
$$
\text{Minimiser} \sum_X \sum_T (P_t^{(X)} - P_{min}^{(X)}N_t^{(X)}) C_{MWh}^{(X)} + N_t^{(X)}C_{base}^{(X)} + N_{start,t}^{(X)}C_{start}^{(X)}
$$

## 3 Réserve de puissance

Pour intégrer la réserve de puissance on ajoute la contrainte :
$$
\sum_X N_t^{(X)}P_{max}^{(X)}\geq D_t\times 1,15
$$

## 4 Planification cyclique

Pour ce problème on modifie la convention $N_{-1}^{(X)}=0$ par $N_{-1}^{(X)}=N_{23}^{(X)}$

## 5 Centrales hydroélectriques

### 5.1

On ajoute les variables de décisions suivantes:<br>
pour $Y\in[9,14]$ (pour les centrales de 900 MW et 1400MW),<br>
$$
H_t^{(Y)} \in \{0,1\} \text{  , vaut 1 si la centrale $X$ fonctionne à } t \text{ h 0 sinon}
$$
$$
H_{start,t}^{(Y)} \in \{0,1\} \text{  , vaut 1  si la centrale $X$ démarre à } t \text{ h 0 sinon}
$$
Avec les contraintes :
$$
H_{start,t}^{(Y)} \leq 1 - H_{t-1}^{(Y)} \text{  ,  Il n'y a pas de démarrage si la centrale est déjà allumée}
$$
$$
H_{start,t}^{(Y)} \leq H_{t}^{(Y)} \text{  ,  La centrale fonctionne lorsqu'elle est est démarrée}
$$
$$
\sum_X P_t^{(X)} + \sum_Y N_t^{(Y)}P^{(Y)}= D_t \text{  ,  Contrainte équilibre offre-demande}
$$
$$
\sum_X N_t^{(X)}P_{max}^{(X)} + \sum_X (1-H_t^{(Y)})P^{(Y)}\geq D_t\times 1,15 \text{  ,  marges de sécurité}
$$
L'objectif mis à jour devient:
$$
\text{Minimiser} \sum_X \sum_t (P_t^{(X)} - P_{min}^{(X)}N_t^{(X)}) C_{MWh}^{(X)} + N_t^{(X)}C_{base}^{(X)} + N_{start,t}^{(X)}C_{start}^{(X)} + \sum_Y \sum_t H_t^{(Y)}C_{base}^{(Y)} + H_t^{(Y)}C_{start}^{(Y)}
$$

### 5.2

On ajoute les variables de décision suivante :
$$
S_t =  \text{Puissance appelée par le pompage à l'instant }t
$$
Puis la contrainte :
$$
S_t d^{(S)} = \sum_Y N_t^{(Y)}d^{(Y)}
$$
Où $d^{(S)}$ représente la hauteur d'eau élevée par MWh et $d^{(Y)}$ la hauteur d'eau prélevée lorsque la centrale $Y$ fonctionne.<br>
On met à jour les contraintes suivante:
$$
\sum_X P_t^{(X)} + \sum_Y N_t^{(Y)}P^{(Y)}-S_t= D_t \text{  ,  Contrainte équilibre offre-demande}
$$